In [ ]:
import importlib
from utils import (
    extraction,
    generic_helper,
    experiment
)
from utils.definitions import ROOT_DIR
importlib.reload(generic_helper)
importlib.reload(extraction)
importlib.reload(experiment)

In [ ]:
# read in train-test split
SPLIT_DATA = generic_helper.read_data(
    path=f"{ROOT_DIR}/data", fname="noah_train_test_split.pkl"
)

# load model configuration
MODEL_CONFIG = generic_helper.load_yaml_file(
    path=f"{ROOT_DIR}/config/model_config.yaml"
)

# load transform functions
TRANSFORM_FUNCTIONS = dict(
    zip(
        ["eol", "rul", "classification"],
        [
            extraction.get_data_for_eol_prediction,
            extraction.get_data_for_rul_prediction,
            extraction.get_data_for_classification,
        ],
    )
)

# define scorers 
SCORERS = dict(
    zip(
        ["eol", "rul", "classification"],
        [
            "neg_mean_absolute_error",
            "neg_mean_absolute_error",
            "f1"
        ],
    )
)

In [ ]:
# define a function that will take model type and run the
# windowed threshold experiment

def windowed_threshold_experiment(
    model_type: str,
) -> tuple:
  
    threshold_result = experiment.effect_time_threshold(
        train_data=SPLIT_DATA["train"],
        signature_depth=MODEL_CONFIG[model_type]["signature_depth"],
        param_grid=MODEL_CONFIG[model_type]["param_grid"],
        problem_type="regression" if model_type in ["rul", "eol"] else "classification",
        trans_func=TRANSFORM_FUNCTIONS[model_type],
        scorer=SCORERS[model_type],
        list_of_threshold=[(0, 40), (40, 80), (80, 120)],
    )

    return threshold_result

In [ ]:
# call the function to see results 
for model_type in SCORERS.keys():

    print(f"Running {model_type} model...")
    print("------------------------------")

    windowed_threshold_experiment(model_type=model_type)
    